<a href="https://colab.research.google.com/github/yigao1056/strongbaseline/blob/master/wheat_of_vgg_16_FC__colab_regression_9scores_mask_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget https://raw.githubusercontent.com/StefOe/colab-pytorch-utils/master/utils.py

--2019-10-16 03:42:54--  https://raw.githubusercontent.com/StefOe/colab-pytorch-utils/master/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4519 (4.4K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]   4.41K  --.-KB/s    in 0s      

2019-10-16 03:42:54 (134 MB/s) - ‘utils.py’ saved [4519/4519]



In [0]:
import torch
#from engine import train_one_epoch, evaluate
from torch.utils.data import Dataset, DataLoader
import utils
import torchvision
#from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torch.utils.data
from PIL import Image, ImageFile
import pandas as pd
from tqdm import tqdm
import collections
import os
import numpy as np
import random
import re
from torchvision import datasets, models
import torchvision.transforms as T2

In [0]:
mean = [0.54652738, 0.54972051, 0.56007025]#crop
std = [0.11290856, 0.11084755, 0.10692802]#crop
#mean = [0.51115957, 0.51408631, 0.52555] # crop_no_mask
#std = [0.08208467, 0.07920669, 0.07619518] # crop_no_mask

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
!mkdir -p drive
!google-drive-ocamlfuse drive 
import os
os.chdir("drive/Colab Notebooks") 

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131183 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.13-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.13-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.13-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [0]:
#define dataloader
class Steel_Dataset(Dataset):
    
    def __init__(self, root,txt,csv,vgg_path='E:/vgg_output_224/', height=0, width=0, transforms=None):
        #1.Load all img as a list
        self.root = root
        #self.transform2=T.ToTensor()
        self.csv = csv
        self.height = height
        self.width = width
        self.vgg_path = vgg_path
        
        fh = open(txt, 'r')
        imgs = []
        for line in fh:
            line = line.strip('\n')
            line = line.strip()
            imgs.append(line)
        self.imgs = imgs
        self.transforms = transforms
        self.resize = T2.Resize((height,width))
    
    def __getitem__(self, idx):
        name_1 = self.root + self.imgs[idx].split(':')[0] + '.jpg'
        name_2 = self.root + self.imgs[idx].split(':')[0]+ '.JPG'
        name = ""
        if(os.path.exists(name_1)):
            name = name_1
        else:
            name = name_2
        #print("Loading : ", name)
        data_transforms = T2.Compose([#T2.RandomResizedCrop(224),
            #T2.RandomHorizontalFlip(),
            T2.Resize((512,512)),
            T2.ToTensor(),
            T2.Normalize(mean, std)
        ])
        #img = self.resize(img)
        #print(type(img))
        image = Image.open(name)
        image = data_transforms(image)
        
        readscv = pd.read_csv(self.csv)
        
        job_num = self.imgs[idx].split()[0]
        index_csv = int(self.imgs[idx].split(":")[1])
        #print("Loading : ", int(self.imgs[idx].split(":")[1]))
        #age = int(readscv.loc[index_csv]["Age"].replace(",",""))
        
        den_scores = [readscv.loc[index_csv]['Dense_Flat'],readscv.loc[index_csv]['Dense_3T'],readscv.loc[index_csv]['Dense_5T']]
        size_scores = [readscv.loc[index_csv]['Size_Flat'],readscv.loc[index_csv]['Size_3T'],readscv.loc[index_csv]['Size_5T']]
        wcp_scores = [readscv.loc[index_csv]['WCP_Flat'],readscv.loc[index_csv]['WCP_3T'],readscv.loc[index_csv]['WCP_5T']]
        #undercutting_edge = [readscv.loc[index_csv]['Bottom Edge'],readscv.loc[index_csv]['Scribe'],readscv.loc[index_csv]['Side Edge']]
        
        den_scores.extend(size_scores)
        den_scores.extend(wcp_scores)
        #target_score = torch.as_tensor(wcp_scores[1], dtype=torch.float)
        
        #target = {}
        # Todo: add age and undercutting?
        #target["age"] = age
        target= den_scores
        target = torch.as_tensor(target)
        #target["blister_size"] = size_scores #[int int int], (for Flat, 3T, 5T)
        #target["wcp"] = wcp_scores #[int int int], (for Flat, 3T, 5T)
        #print("data loader : target = ",target)

        return image,target
    
    def __len__(self):
        return len(self.imgs)        

In [0]:
# control panel
_root = 'drive/black_mask_background/''#'drive/crop/'#'E:/crop/'
_num_of_epoch = 180
_batch_size_train = 10
_batch_size_test = 2
_height = 512
_width = 512
_txt_train = 'drive/train.txt'
_txt_test = 'drive/test.txt'
_csv_file = 'drive/training_data.csv'
_learning_rate = 0.001
_print_freq = 1
_device = 'cpu'
vgg16_net = models.vgg16(pretrained=False)

In [0]:
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from sklearn.metrics import accuracy_score
from torch.utils.tensorboard import SummaryWriter

# define training model
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv = vgg16_net.features
        
        #nn.Sequential(
         #   nn.Conv2d(3,6,5),
          #  nn.MaxPool2d(2, 2),
           # nn.ReLU(),
            #nn.Conv2d(6,16,5),
            #nn.MaxPool2d(4, 4),
            #nn.ReLU(),
            #nn.Conv2d(16,32,5),
            #nn.MaxPool2d(4, 4),
        #)
        self.fc = nn.Sequential(
            nn.Linear(512*16*16,512),
            nn.ReLU(),
            nn.Linear(512,9)
        )
        self.loss_func = nn.MSELoss()

    def forward(self, x):
        #n=[]
        #for img in x:            
        #    n.append(img.detach().numpy())
        #n = torch.as_tensor(n,dtype=torch.float)
        n = self.conv(x)
        #print(n.shape)
        n = n.view(-1, self.num_flat_features(n))
        n = self.fc(n)
        return n

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


In [0]:
import torch.optim as optim
#vgg16_net = models.vgg16(pretrained=False)
model = Net()
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=_learning_rate, momentum=0.9)
train_data = Steel_Dataset( _root, _txt_train, _csv_file, height=_height, width=_width, transforms=None )
train_loader = DataLoader(dataset=train_data, batch_size = _batch_size_train, shuffle=True,num_workers=0,collate_fn=None)
test_data = Steel_Dataset(_root, _txt_test, _csv_file,transforms= None)
test_loader = DataLoader(dataset=test_data, batch_size =10, shuffle=True)

In [0]:
from sklearn.metrics import accuracy_score
writer = SummaryWriter()
writer_index = 0
for epoch in range(_num_of_epoch):  # loop over the dataset multiple times
    running_loss = 0.0
    
    pre_out = []
    label_out = []
    acc = 0
    for j, data in enumerate(test_loader,1):
        print("Test: loading batch No.",j,"...")
        image, labels = data
        #print(labels)
        image = image.cuda()
        model.cuda()
        outputs = model(image)
        #print("train : ",outputs[0].shape,labels[0].shape)
        #print("train : labels = ",labels)
        outputs = outputs.cpu()

        for out in outputs:
            pre_out.append(np.rint(out.detach().numpy()))
        
        for label in labels:
            label_out.append(label.detach().numpy())
        
    pre_out = np.rint(np.array(pre_out)).astype(int)
    label_out = np.rint(np.array(label_out)).astype(int)
    
    acc=[]
    for j in range(9):
        ytrue = label_out[:,j]
        ypred = pre_out[:,j]
        #print(ytrue,ypred)
        acc.append(accuracy_score(ytrue,ypred))
    
    print('epoch : ',epoch ,'accuracy_score is : ',acc)
    #writer.add_scalar('Own_Conv_Fc/Accurancy',acc[0], epoch) #add data to tensorboard
    writer.add_scalar('VGG16_Conv_Fc/Accurancy_dens_flat',acc[0],epoch)
    writer.add_scalar('VGG16_Conv_Fc/Accurancy_dens_3t',acc[1],epoch)
    writer.add_scalar('VGG16_Conv_Fc/Accurancy_dens_5t',acc[2],epoch)
    writer.add_scalar('VGG16_Conv_Fc/Accurancy_size_flat',acc[3],epoch)
    writer.add_scalar('VGG16_Conv_Fc/Accurancy_size_3t',acc[4],epoch)
    writer.add_scalar('VGG16_Conv_Fc/Accurancy_size_5t',acc[5],epoch)
    writer.add_scalar('VGG16_Conv_Fc/Accurancy_wcp_flat',acc[6],epoch)
    writer.add_scalar('VGG16_Conv_Fc/Accurancy_wcp_3t',acc[7],epoch)
    writer.add_scalar('VGG16_Conv_Fc/Accurancy_wcp_5t',acc[8],epoch)

    
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, scores = data
        #print(inputs[0].shape)
        
        # zero the parameter gradients
        optimizer.zero_grad()
        inputs = inputs.cuda()
        model.cuda()
        # forward + backward + optimize
        outputs = model(inputs)
        
        score_tensor = []
        for score in scores:
            score_tensor.append(score.detach().numpy())
        score_tensor = torch.as_tensor(score_tensor)   
        score_tensor = score_tensor.cuda()
        loss = criterion(outputs, score_tensor)
        writer_index = i+epoch*len(train_loader)
        writer.add_scalar('VGG16_Conv_Fc/Loss', loss, writer_index) #add data to tensorboard
        loss.backward()
        optimizer.step()
        
        print('epoch', epoch, '[',i,'/',len(train_loader),']','Loss is:',loss.detach())
        
print('Finished Training')
writer.close()

Test: loading batch No. 1 ...
Test: loading batch No. 2 ...
Test: loading batch No. 3 ...
Test: loading batch No. 4 ...
Test: loading batch No. 5 ...
Test: loading batch No. 6 ...
Test: loading batch No. 7 ...
Test: loading batch No. 8 ...
Test: loading batch No. 9 ...
Test: loading batch No. 10 ...
Test: loading batch No. 11 ...
Test: loading batch No. 12 ...
Test: loading batch No. 13 ...
Test: loading batch No. 14 ...
Test: loading batch No. 15 ...
Test: loading batch No. 16 ...
Test: loading batch No. 17 ...
Test: loading batch No. 18 ...
Test: loading batch No. 19 ...
Test: loading batch No. 20 ...
Test: loading batch No. 21 ...
Test: loading batch No. 22 ...
Test: loading batch No. 23 ...
epoch :  0 accuracy_score is :  [0.631578947368421, 0.17982456140350878, 0.2675438596491228, 0.7105263157894737, 0.17982456140350878, 0.2675438596491228, 0.9473684210526315, 0.008771929824561403, 0.17543859649122806]
epoch 0 [ 0 / 54 ] Loss is: tensor(4.2692, device='cuda:0')
epoch 0 [ 1 / 54 ] 

FileNotFoundError: ignored

In [0]:
torch.save(model, 'drive/model.bin')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [0]:
torch.save(model.state_dict(),  'drive/model_dict.pt')

In [0]:
print(accuracy_score(label_out, pre_out))